In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Obtain Moons

In [2]:
list_of_moons_page = requests.get('https://littleastronomy.com/moon-names/')
moon_more_info_page = requests.get('https://courses.lumenlearning.com/suny-astronomy/chapter/selected-moons-of-the-planets/')
list_of_planets_page = requests.get('https://en.wikipedia.org/wiki/Planet#Solar_System');

In [3]:
soup = BeautifulSoup(list_of_moons_page.text, 'lxml')

# Initialize the table
headers = ['name', 'planet', 'name_meaning']
df_moon = pd.DataFrame(columns=headers)

# Enter data for each row into the dataframe
table = soup.find("table")
for table_row in table.find_all('tr')[1:]:
  data = [table_data.text.strip() for table_data in table_row.find_all('td')]
  df_moon.loc[len(df_moon)] = data

# Display the table
df_moon.head()

,name,planet,name_meaning
0,Moon,Earth,“month”
1,Phobos,Mars,Greek god of fear and panic
2,Deimos,Mars,Greek god of dread and terror
3,Io,Jupiter,"Ancestor to Perseus, Cadmus, Heracles, and Minos"
4,Europa,Jupiter,Mother of king Minos of Crete


In [4]:
def convert_string_to_integer(str):
    str_integer = ''.join([n for n in str if n.isdigit()])
    return None if str_integer == '' else int(str_integer)

In [5]:
soup = BeautifulSoup(moon_more_info_page.text, 'lxml')

# Initialize the table
headers = ['name', 'year_discovered', 'diameter_in_km']
df_moon_additional_info  = pd.DataFrame(columns=headers)

# Enter data for each row into the dataframe
table = soup.find("table")
for table_row in table.find_all('tr')[2:]:
  rows = table_row.find_all('td')
  data = np.array([table_data.text.strip() for table_data in rows])
  selected_data = data[[1, 2, 5]] if len(data) == 8 else data[[0, 1, 4]]
  selected_data[1] = convert_string_to_integer(selected_data[1])
  table_idx = len(df_moon_additional_info)
  df_moon_additional_info.loc[table_idx] = selected_data

df_moon_additional_info.replace(to_replace='None', value=np.nan, inplace=True)

# Display the table
df_moon_additional_info.head()

,name,year_discovered,diameter_in_km
0,Moon,NaN,3476
1,Phobos,1877,23
2,Deimos,1877,13
3,Amalthea,1892,200
4,Thebe,1979,90


In [6]:
# outer join
df_moon_final = pd.merge(df_moon, df_moon_additional_info, on='name', how='outer')

In [7]:
# check if there are null values
print("Null values: ", df_moon_final.isnull().sum().sum())

# remove rows with null values
df_moon_final.dropna(inplace=True)

# check there are no more null values
print("Null values: ", df_moon_final.isnull().sum().sum())

Null values:  381
Null values:  0


In [8]:
df_moon_final.head(6)

,name,planet,name_meaning,year_discovered,diameter_in_km
1,Phobos,Mars,Greek god of fear and panic,1877,23
2,Deimos,Mars,Greek god of dread and terror,1877,13
3,Io,Jupiter,"Ancestor to Perseus, Cadmus, Heracles, and Minos",1610,3630
4,Europa,Jupiter,Mother of king Minos of Crete,1610,3138
5,Ganymede,Jupiter,Male hero of Troy. Served as Zeus cup-bearer i...,1610,5262
6,Callisto,Jupiter,One of the nymphs. Daughter of king Lycaon and...,1610,4800


In [9]:
must_have_planets = df_moon_final.planet.unique() 
print(must_have_planets)

['Mars' 'Jupiter' 'Saturn' 'Uranus' 'Neptune']


Missing one must have planet. So let's manually add a moon related to the planet.

In [10]:
df_moon_final.loc[len(df_moon_final)] = ['Nix', 'Pluto', 'Geek Goddess of the night', 2005, 46]

In [11]:
must_have_planets = df_moon_final.planet.unique() 
print(must_have_planets)

['Mars' 'Jupiter' 'Saturn' 'Uranus' 'Neptune' 'Pluto']


# Obtain Planets

In [12]:
def remove_non_alpha(str):
    return ''.join([i for i in str if i.isalpha() or i.isspace()]);

In [13]:
soup = BeautifulSoup(list_of_planets_page.text, 'lxml')

# Initialize the table
headers = ['name', 'orbital_period_in_years', 'rotation_period_in_days', 'has_moon']
df_planets = pd.DataFrame(columns=headers)

# Enter data for each row into the dataframe
table = soup.find_all("table")[4]
for table_row in table.find_all('tr')[2:]:
  rows = table_row.find_all('td')[1:]
  data = np.array([table_data.text.strip() for table_data in rows])
  
  if len(data) == 0:
    continue
  
  selected_data = data[[0, 4, 7, 8]]
  selected_data[0] = remove_non_alpha(selected_data[0]) # clean string
  df_planets.loc[len(df_planets)] = selected_data

# Replace '?' with NaN
df_planets = df_planets.replace('?', np.NaN)

# Modify the values in "has_moon" column
df_planets['has_moon'] = df_planets['has_moon'].astype(int)
df_planets['has_moon'] = df_planets['has_moon'].apply(np.sign).replace({0: False, 1: True})

# Display the table
df_planets

,name,orbital_period_in_years,rotation_period_in_days,has_moon
0,Mercury,0.24,58.65,False
1,Venus,0.62,243.02,False
2,Earth,1.00,1.00,True
3,Mars,1.88,1.03,True
4,Jupiter,11.86,0.41,True
5,Saturn,29.45,0.44,True
6,Uranus,84.02,0.72,True
7,Neptune,164.79,0.67,True
8,Ceres,4.60,0.38,False
9,Orcus,247.5,NaN,True


## Check if all must have planets are present

In [14]:
for planet_name in must_have_planets: 
    if planet_name not in df_planets.name.values: 
        print(planet_name)

In [15]:
# Add the "planet_id" column
df_planets.reset_index(inplace=True)
df_planets.rename(columns={'index': 'planet_id'}, inplace=True)
df_planets['planet_id'] += 1

In [16]:
df_planets.tail()

,planet_id,name,orbital_period_in_years,rotation_period_in_days,has_moon
12,13,Quaoar,288.0,0.74,True
13,14,Makemake,306.2,0.95,True
14,15,Gonggong,552.5,0.93,True
15,16,Eris,559,15.79,True
16,17,Sedna,12059,0.43,False


# Creating the first table: moon table (final)

In [17]:
# Merge moon table and planet table, then match the planet's id and name
df_moon_merged = pd.merge(df_moon_final, df_planets[['planet_id', 'name']], 
                   left_on='planet', right_on='name', 
                   how='inner')

# Drop "planet" and "name_y" columns
df_moon_merged.drop(['planet', 'name_y'], axis=1, inplace=True)

# Change "name_x" column to "name" column
df_moon_merged.rename(columns={'name_x': 'name'}, inplace=True)

# Display the first five rows of the final moon table
df_moon_merged.head()

,name,name_meaning,year_discovered,diameter_in_km,planet_id
0,Phobos,Greek god of fear and panic,1877,23,4
1,Deimos,Greek god of dread and terror,1877,13,4
2,Io,"Ancestor to Perseus, Cadmus, Heracles, and Minos",1610,3630,5
3,Europa,Mother of king Minos of Crete,1610,3138,5
4,Ganymede,Male hero of Troy. Served as Zeus cup-bearer i...,1610,5262,5


# Creating constellation and star tables

The constellation that the planet is "in".

[1] The Sky Live, Major Solar System Objects, https://theskylive.com/planets

[2] The Sky Live, Asteroids, https://theskylive.com/asteroids-and-dwarf-planets

[3] 50000 Quaoar. https://usuaris.tinet.cat/klunn/quaoar.html

[4] Wikipedia, 225088 Gonggong, https://en.wikipedia.org/wiki/225088_Gonggong

In [18]:
planet_constellation = {
    'Mercury' : 'Virgo', #1
    'Venus' : 'Leo', #1
    'Earth' : 'Centaurus',
    'Mars' : 'Taurus', #!
    'Jupiter' : 'Cetus', #!
    'Saturn' : 'Capricornus', #!
    'Uranus' : 'Aries', #!
    'Neptune' : 'Aquarius', #!
    'Ceres' : 'Leo', #2
    'Orcus' : 'Sagittarius', 
    'Pluto' : 'Sagittarius', #2 
    'Haumea' : 'Boötes', #2
    'Quaoar' : 'Ophiuchus', #3
    'Makemake' : 'Coma Berenices', #2 
    'Gonggong' : 'Aquarius', #4
    'Eris' : 'Cetus', #2
    'Sedna' : 'Taurus', #2
    'Salacia' : 'Pegasus', 
    'Varda' : 'Aquarius'
}

In [19]:
# Turn the dictionary into a table
df_planet_constellation = pd.DataFrame.from_dict(planet_constellation, orient='index').reset_index()
df_planet_constellation.columns = ['planet', 'constellation']
df_planet_constellation

,planet,constellation
0,Mercury,Virgo
1,Venus,Leo
2,Earth,Centaurus
3,Mars,Taurus
4,Jupiter,Cetus
5,Saturn,Capricornus
6,Uranus,Aries
7,Neptune,Aquarius
8,Ceres,Leo
9,Orcus,Sagittarius


## Scraping constellation and its brightest star

In [20]:
# Get the table from Wikipedia page
page = requests.get('https://en.wikipedia.org/wiki/IAU_designated_constellations')
soup = BeautifulSoup(page.text, 'lxml')
table = soup.find('table', {'class':"wikitable"})
df_constellation = pd.read_html(str(table))
df_constellation = pd.DataFrame(df_constellation[0])

# Choose the selected columns in the table
df_constellation = df_constellation[['Constellation', 'Origin', 'Meaning', 'Brightest star']]
df_constellation.columns = ['name', 'year_discovered', 'meaning', 'star']

# cleaning up the data in the table
df_constellation['name'] = [word.split('/')[0].strip() for word in df_constellation['name']]
df_constellation['year_discovered'] = df_constellation['year_discovered'].replace('ancient (Ptolemy)', '')
df_constellation['year_discovered'] = [year.split(',')[0] for year in df_constellation['year_discovered']]
df_constellation['meaning'] = df_constellation['meaning'].str.capitalize()

# Display the table
df_constellation

,name,year_discovered,meaning,star
0,Andromeda,,Andromeda (the chained maiden or princess),Alpheratz
1,Antlia,1763,Air pump,α Antliae
2,Apus,1603,Bird-of-paradise/exotic bird/extraordinary bird,α Apodis
3,Aquarius,,Water-bearer,Sadalsuud
4,Aquila,,Eagle,Altair
...,...,...,...,...
83,Ursa Minor,,Lesser bear,Polaris
84,Vela,1763,Sails,γ2 Velorum
85,Virgo,,Virgin or maiden,Spica
86,Volans,1603,Flying fish,β Volantis


In [21]:
df_constellation.star.values

array(['Alpheratz', 'α Antliae', 'α Apodis', 'Sadalsuud', 'Altair',
       'β Arae', 'Hamal', 'Capella', 'Arcturus', 'α Caeli',
       'β\xa0Camelopardalis', 'Tarf[9]', 'Cor Caroli', 'Sirius',
       'Procyon', 'Deneb Algedi', 'Canopus', 'Schedar[9]',
       'Rigil Kentaurus[9]', 'Alderamin', 'Diphda[9]', 'α Chamaeleontis',
       'α Circini', 'Phact', 'β Comae Berenices', 'Meridiana[9]',
       'Alphecca', 'Gienah', 'δ Crateris', 'Acrux', 'Deneb', 'Rotanev',
       'α Doradus', 'Eltanin[9]', 'Kitalpha', 'Achernar', 'Dalim[9]',
       'Pollux', 'Alnair', 'Kornephoros', 'α Horologii', 'Alphard',
       'β Hydri', 'α Indi', 'α Lacertae', 'Regulus', 'Praecipua', 'Arneb',
       'Zubeneschamali[9]', 'α Lupi', 'α Lyncis', 'Vega', 'α Mensae',
       'γ Microscopii', 'β Monocerotis', 'α Muscae', 'γ2 Normae',
       'ν Octantis', 'Rasalhague', 'Rigel', 'Peacock', 'Enif', 'Mirfak',
       'Ankaa', 'α Pictoris', 'Alpherg', 'Fomalhaut', 'Naos', 'α Pyxidis',
       'α Reticuli', 'γ Sagittae', 'Kau

Out of practicality, I would like to remove the star column’s use of Greek letters on their records to our alphabet. And if you check further, the star’s name also contains [9]s, so I’ll omit them as well.

In [22]:
df_constellation['star'] = df_constellation['star'].apply(remove_non_alpha)

In [23]:
df_constellation.star.values

array(['Alpheratz', 'α Antliae', 'α Apodis', 'Sadalsuud', 'Altair',
       'β Arae', 'Hamal', 'Capella', 'Arcturus', 'α Caeli',
       'β\xa0Camelopardalis', 'Tarf', 'Cor Caroli', 'Sirius', 'Procyon',
       'Deneb Algedi', 'Canopus', 'Schedar', 'Rigil Kentaurus',
       'Alderamin', 'Diphda', 'α Chamaeleontis', 'α Circini', 'Phact',
       'β Comae Berenices', 'Meridiana', 'Alphecca', 'Gienah',
       'δ Crateris', 'Acrux', 'Deneb', 'Rotanev', 'α Doradus', 'Eltanin',
       'Kitalpha', 'Achernar', 'Dalim', 'Pollux', 'Alnair', 'Kornephoros',
       'α Horologii', 'Alphard', 'β Hydri', 'α Indi', 'α Lacertae',
       'Regulus', 'Praecipua', 'Arneb', 'Zubeneschamali', 'α Lupi',
       'α Lyncis', 'Vega', 'α Mensae', 'γ Microscopii', 'β Monocerotis',
       'α Muscae', 'γ Normae', 'ν Octantis', 'Rasalhague', 'Rigel',
       'Peacock', 'Enif', 'Mirfak', 'Ankaa', 'α Pictoris', 'Alpherg',
       'Fomalhaut', 'Naos', 'α Pyxidis', 'α Reticuli', 'γ Sagittae',
       'Kaus Australis', 'Antares', 

## Create the table for planet and its related star

In [24]:
df_planet_star = pd.merge(df_planet_constellation, 
                          df_constellation[['name', 'star']], 
                          how='inner', 
                          left_on='constellation', 
                          right_on='name')

df_planet_star = df_planet_star[['planet', 'star']]
df_planet_star = df_planet_star.rename(columns={'planet': 'name'})
df_planet_star

,name,star
0,Mercury,Spica
1,Venus,Regulus
2,Ceres,Regulus
3,Earth,Rigil Kentaurus
4,Mars,Aldebaran
5,Sedna,Aldebaran
6,Jupiter,Diphda
7,Eris,Diphda
8,Saturn,Deneb Algedi
9,Uranus,Hamal


## Keep the Relevant Conestellations

In [25]:
keep_constellation = df_planet_constellation.constellation.unique() 
keep_constellation

array(['Virgo', 'Leo', 'Centaurus', 'Taurus', 'Cetus', 'Capricornus',
       'Aries', 'Aquarius', 'Sagittarius', 'Boötes', 'Ophiuchus',
       'Coma Berenices', 'Pegasus'], dtype=object)

In [26]:
df_constellation = df_constellation[df_constellation.name.isin(keep_constellation)]
df_constellation = df_constellation.reset_index(drop=True)
df_constellation

,name,year_discovered,meaning,star
0,Aquarius,,Water-bearer,Sadalsuud
1,Aries,,Ram,Hamal
2,Boötes,,Herdsman,Arcturus
3,Capricornus,,Sea goat,Deneb Algedi
4,Centaurus,,Centaur,Rigil Kentaurus
5,Cetus,,Sea monster (later interpreted as a whale),Diphda
6,Coma Berenices,1536,Berenice's hair,β Comae Berenices
7,Leo,,Lion,Regulus
8,Ophiuchus,,Serpent-bearer,Rasalhague
9,Pegasus,,Pegasus (mythological winged horse),Enif


## Creating the Star Table

We will input the data manually as data scrapping will be too tedius.

In [27]:
star_dict = {
    'star_id' : list(range(1, 14)),
    'name' : ['Sadalsuud', 'Hamal', 'Arcturus', 'Deneb Algedi', 'Rigil Kentaurus', 'β Comae Berenices', 
              'Regulus', 'Aldebaran', 'Diphda', 'Enif', 'Kaus Australis', 'Rasalhague', 'Spica'],
    'distance_in_light_years' : [612.00, 66, 37.00, 39, 4.36, 30.00, 77.00, 
                                 65.23, 96.22, 688.2, 143.2, 48.6, 260.9],
    'brightness_of_star' : [2.90, 2.01, 0.05, 2.85, 0.01, 4.26, 1.36, 
                            0.85, 2.04, 2.4, 1.85, 2.08, 1.04],
    'approval_date' : ['2016-08-21', '2016-07-20', '2016-06-30', '2017-02-01', '2016-11-06', np.NaN, 
                       '2016-06-30', '2016-06-30', '2016-08-21', '2016-07-20', '2016-07-20', '2016-07-20', '2016-06-30'],
    'galaxy' : ['Milky Way'] * 13
}

df_star = pd.DataFrame.from_dict(star_dict)
df_star

,star_id,name,distance_in_light_years,brightness_of_star,approval_date,galaxy
0,1,Sadalsuud,612.00,2.90,2016-08-21,Milky Way
1,2,Hamal,66.00,2.01,2016-07-20,Milky Way
2,3,Arcturus,37.00,0.05,2016-06-30,Milky Way
3,4,Deneb Algedi,39.00,2.85,2017-02-01,Milky Way
4,5,Rigil Kentaurus,4.36,0.01,2016-11-06,Milky Way
5,6,β Comae Berenices,30.00,4.26,NaN,Milky Way
6,7,Regulus,77.00,1.36,2016-06-30,Milky Way
7,8,Aldebaran,65.23,0.85,2016-06-30,Milky Way
8,9,Diphda,96.22,2.04,2016-08-21,Milky Way
9,10,Enif,688.20,2.40,2016-07-20,Milky Way


# Creating the final planet table

We can add a star_id column to the planet table now that we have (almost) completed the star table. Later, we will also replace the galaxy column in this star table with the galaxy_id.

In [28]:
# Add the star column to the planet table
df_planets = pd.merge(df_planets, df_planet_star, how='inner', on='name')

# Add the appropriate star_id to the planet table
df_planets = pd.merge(df_planets, df_star[['star_id', 'name']], how='inner', left_on='star', right_on='name')

# Omit the columns that are not needed
df_planets = df_planets.drop(columns=['star', 'name_y'])

# Change back the name of planet column to "name"
df_planets = df_planets.rename(columns={'name_x' : 'name'})

# Sort planet_id column and reset the index
df_planets = df_planets.sort_values(by='planet_id').reset_index(drop=True)

# Display the table
df_planets

,planet_id,name,orbital_period_in_years,rotation_period_in_days,has_moon,star_id
0,1,Mercury,0.24,58.65,False,13
1,2,Venus,0.62,243.02,False,7
2,3,Earth,1.00,1.00,True,5
3,4,Mars,1.88,1.03,True,8
4,5,Jupiter,11.86,0.41,True,9
5,6,Saturn,29.45,0.44,True,4
6,7,Uranus,84.02,0.72,True,2
7,8,Neptune,164.79,0.67,True,1
8,9,Ceres,4.60,0.38,False,7
9,10,Orcus,247.5,NaN,True,11


# Creating the constellation table (final)

In [29]:

# Add the appropriate star_id to the constellation table
df_constellation = pd.merge(df_constellation, df_star[['star_id', 'name']], how='inner', left_on='star', right_on='name')

# Omit the columns that are not needed
df_constellation = df_constellation.drop(columns=['star', 'name_y'])

# Change back the name of planet column to "name"
df_constellation = df_constellation.rename(columns={'name_x' : 'name'})

# Add the "constellation_id" column
df_constellation.reset_index(inplace=True)
df_constellation.rename(columns={'index': 'constellation_id'}, inplace=True)
df_constellation['constellation_id'] += 1

# Display the table
df_constellation

,constellation_id,name,year_discovered,meaning,star_id
0,1,Aquarius,,Water-bearer,1
1,2,Aries,,Ram,2
2,3,Boötes,,Herdsman,3
3,4,Capricornus,,Sea goat,4
4,5,Centaurus,,Centaur,5
5,6,Cetus,,Sea monster (later interpreted as a whale),9
6,7,Coma Berenices,1536,Berenice's hair,6
7,8,Leo,,Lion,7
8,9,Ophiuchus,,Serpent-bearer,12
9,10,Pegasus,,Pegasus (mythological winged horse),10


# Creating the last table: galaxy table (final)

In [30]:
page = requests.get('https://littleastronomy.com/galaxy-names/')
soup = BeautifulSoup(page.text, 'lxml')

# Initialize the table
headers = ['name', 'designation', 'constellation', 'meaning']
df_galaxy = pd.DataFrame(columns = headers)

# Enter data for each row into the dataframe
table = soup.find("table")
for table_row in table.find_all('tr')[1:]:
  data = [table_data.text.strip() for table_data in table_row.find_all('td')]
  df_galaxy.loc[len(df_galaxy)] = data

# Delete "designation" column
df_galaxy = df_galaxy.drop(columns='designation')

# Add the contellation (center) of Milky Way 
df_galaxy.loc[26, 'constellation'] = 'Sagittarius'

# Show a snippet of the table.
df_galaxy.head(10)

,name,constellation,meaning
0,Andromeda,Andromeda,"In mythology, Andromeda is the daughter of the..."
1,Antennae Galaxy,Corvus,This is a dual galaxy. It gets its name becaus...
2,Backward Galaxy,Centaurus,It seems to rotate in the opposite direction t...
3,Black Eye Galaxy,Coma Berenices,It looks like an eye with a dark stripe undern...
4,Bode’s Galaxy,Ursa Major,"Named after the astronomer who discovered it, ..."
5,Butterfly Galaxies,Virgo,Binary galaxies. It looks like a pair of butte...
6,Cartwheel Galaxy,Sculptor,It looks a bit like a cartwheel
7,Centaurus A,Centaurus,Named because it’s located in the Centaurus co...
8,Cigar Galaxy,Ursa Major,It is shaped like a cigar
9,Circinus,Circinus,Latin for compass. Named after the constellati...


In [31]:
kept_constellation = df_constellation.name.values
df_galaxy = df_galaxy[df_galaxy.constellation.isin(kept_constellation)].reset_index(drop=True)
df_galaxy

,name,constellation,meaning
0,Backward Galaxy,Centaurus,It seems to rotate in the opposite direction t...
1,Black Eye Galaxy,Coma Berenices,It looks like an eye with a dark stripe undern...
2,Butterfly Galaxies,Virgo,Binary galaxies. It looks like a pair of butte...
3,Centaurus A,Centaurus,Named because it’s located in the Centaurus co...
4,Coma Pinwheel Galaxy,Coma Berenices,It looks like a paper pinwheel
5,Little Sombrero Galaxy,Pegasus,"It looks like a sombrero, but it’s smaller tha..."
6,Malin 1,Coma Berenices,"Named after its discoverer, David Malin"
7,Mice Galaxies,Coma Berenices,Two galaxies with long tails that look like a ...
8,Milky Way,Sagittarius,Our own galaxy. It is said to look like a band...
9,Needle Galaxy,Coma Berenices,Named because of its thin appearance


In [32]:
visible_galaxy = ['Milky Way', 'Large Magellanic Cloud', 'Small Magellanic Cloud', 'Andromeda', 
                  'Triangulum Galaxy', 'Centaurus A', 'Bode’s Galaxy', 'Sculptor Galaxy']
df_galaxy['is_visible_to_naked_eye'] = df_galaxy.apply(lambda row : True if row['name'] in visible_galaxy else False, axis=1)
df_galaxy

,name,constellation,meaning,is_visible_to_naked_eye
0,Backward Galaxy,Centaurus,It seems to rotate in the opposite direction t...,False
1,Black Eye Galaxy,Coma Berenices,It looks like an eye with a dark stripe undern...,False
2,Butterfly Galaxies,Virgo,Binary galaxies. It looks like a pair of butte...,False
3,Centaurus A,Centaurus,Named because it’s located in the Centaurus co...,True
4,Coma Pinwheel Galaxy,Coma Berenices,It looks like a paper pinwheel,False
5,Little Sombrero Galaxy,Pegasus,"It looks like a sombrero, but it’s smaller tha...",False
6,Malin 1,Coma Berenices,"Named after its discoverer, David Malin",False
7,Mice Galaxies,Coma Berenices,Two galaxies with long tails that look like a ...,False
8,Milky Way,Sagittarius,Our own galaxy. It is said to look like a band...,True
9,Needle Galaxy,Coma Berenices,Named because of its thin appearance,False


In [33]:
# Add the appropriate constellation_id to the galaxy table
df_galaxy = pd.merge(df_galaxy, 
                     df_constellation[['constellation_id', 'name']], 
                     how='inner', 
                     left_on='constellation', 
                     right_on='name')

# Omit the columns that are not needed
df_galaxy = df_galaxy.drop(columns=['constellation', 'name_y'])

# Change back the name of planet column to "name"
df_galaxy = df_galaxy.rename(columns={'name_x' : 'name'})

# Add the "galaxy_id" column
df_galaxy.reset_index(inplace=True)
df_galaxy.rename(columns={'index': 'galaxy_id'}, inplace=True)
df_galaxy['galaxy_id'] += 1

# Display the table
df_galaxy

,galaxy_id,name,meaning,is_visible_to_naked_eye,constellation_id
0,1,Backward Galaxy,It seems to rotate in the opposite direction t...,False,5
1,2,Centaurus A,Named because it’s located in the Centaurus co...,True,5
2,3,Black Eye Galaxy,It looks like an eye with a dark stripe undern...,False,7
3,4,Coma Pinwheel Galaxy,It looks like a paper pinwheel,False,7
4,5,Malin 1,"Named after its discoverer, David Malin",False,7
5,6,Mice Galaxies,Two galaxies with long tails that look like a ...,False,7
6,7,Needle Galaxy,Named because of its thin appearance,False,7
7,8,Butterfly Galaxies,Binary galaxies. It looks like a pair of butte...,False,13
8,9,Sombrero Galaxy,Looks like a sombrero,False,13
9,10,Little Sombrero Galaxy,"It looks like a sombrero, but it’s smaller tha...",False,10


# Final Tables

In [34]:
df_star['galaxy_id'] = [11] * len(df_star) 
df_star = df_star.drop(columns='galaxy') 

In [35]:
df_constellation

,constellation_id,name,year_discovered,meaning,star_id
0,1,Aquarius,,Water-bearer,1
1,2,Aries,,Ram,2
2,3,Boötes,,Herdsman,3
3,4,Capricornus,,Sea goat,4
4,5,Centaurus,,Centaur,5
5,6,Cetus,,Sea monster (later interpreted as a whale),9
6,7,Coma Berenices,1536,Berenice's hair,6
7,8,Leo,,Lion,7
8,9,Ophiuchus,,Serpent-bearer,12
9,10,Pegasus,,Pegasus (mythological winged horse),10


In [36]:
df_galaxy

,galaxy_id,name,meaning,is_visible_to_naked_eye,constellation_id
0,1,Backward Galaxy,It seems to rotate in the opposite direction t...,False,5
1,2,Centaurus A,Named because it’s located in the Centaurus co...,True,5
2,3,Black Eye Galaxy,It looks like an eye with a dark stripe undern...,False,7
3,4,Coma Pinwheel Galaxy,It looks like a paper pinwheel,False,7
4,5,Malin 1,"Named after its discoverer, David Malin",False,7
5,6,Mice Galaxies,Two galaxies with long tails that look like a ...,False,7
6,7,Needle Galaxy,Named because of its thin appearance,False,7
7,8,Butterfly Galaxies,Binary galaxies. It looks like a pair of butte...,False,13
8,9,Sombrero Galaxy,Looks like a sombrero,False,13
9,10,Little Sombrero Galaxy,"It looks like a sombrero, but it’s smaller tha...",False,10


In [37]:
df_star

,star_id,name,distance_in_light_years,brightness_of_star,approval_date,galaxy_id
0,1,Sadalsuud,612.00,2.90,2016-08-21,11
1,2,Hamal,66.00,2.01,2016-07-20,11
2,3,Arcturus,37.00,0.05,2016-06-30,11
3,4,Deneb Algedi,39.00,2.85,2017-02-01,11
4,5,Rigil Kentaurus,4.36,0.01,2016-11-06,11
5,6,β Comae Berenices,30.00,4.26,NaN,11
6,7,Regulus,77.00,1.36,2016-06-30,11
7,8,Aldebaran,65.23,0.85,2016-06-30,11
8,9,Diphda,96.22,2.04,2016-08-21,11
9,10,Enif,688.20,2.40,2016-07-20,11


In [41]:
df_star.to_excel('star.xlsx')

In [38]:
df_planets

,planet_id,name,orbital_period_in_years,rotation_period_in_days,has_moon,star_id
0,1,Mercury,0.24,58.65,False,13
1,2,Venus,0.62,243.02,False,7
2,3,Earth,1.00,1.00,True,5
3,4,Mars,1.88,1.03,True,8
4,5,Jupiter,11.86,0.41,True,9
5,6,Saturn,29.45,0.44,True,4
6,7,Uranus,84.02,0.72,True,2
7,8,Neptune,164.79,0.67,True,1
8,9,Ceres,4.60,0.38,False,7
9,10,Orcus,247.5,NaN,True,11


In [85]:
sql_command = 'INSERT INTO planet(name, orbital_period_in_years, rotation_period_in_days, has_moon, star_id) VALUES'

for i in range(len(df_planets)):
    planet = df_planets.loc[i]
    sql_command += "('" + planet['name'] + "', " + str(planet['orbital_period_in_years']) + ', ' + str(planet['rotation_period_in_days']) + ', ' + str(planet['has_moon']) + ', ' + str(planet['star_id'])
    if i == len(df_planets) - 1: 
        sql_command += ');'
    else:
        sql_command += '), '

print(sql_command);

INSERT INTO planet(name, orbital_period_in_years, rotation_period_in_days, has_moon, star_id) VALUES('Mercury', 0.24, 58.65, False, 13), ('Venus', 0.62, 243.02, False, 7), ('Earth', 1.00, 1.00, True, 5), ('Mars', 1.88, 1.03, True, 8), ('Jupiter', 11.86, 0.41, True, 9), ('Saturn', 29.45, 0.44, True, 4), ('Uranus', 84.02, 0.72, True, 2), ('Neptune', 164.79, 0.67, True, 1), ('Ceres', 4.60, 0.38, False, 7), ('Orcus', 247.5, nan, True, 11), ('Pluto', 247.9, 6.39, True, 11), ('Haumea', 283.8, 0.16, True, 3), ('Quaoar', 288.0, 0.74, True, 12), ('Makemake', 306.2, 0.95, True, 6), ('Gonggong', 552.5, 0.93, True, 1), ('Eris', 559, 15.79, True, 9), ('Sedna', 12059, 0.43, False, 8);


In [39]:
df_moon_merged

,name,name_meaning,year_discovered,diameter_in_km,planet_id
0,Phobos,Greek god of fear and panic,1877,23,4
1,Deimos,Greek god of dread and terror,1877,13,4
2,Io,"Ancestor to Perseus, Cadmus, Heracles, and Minos",1610,3630,5
3,Europa,Mother of king Minos of Crete,1610,3138,5
4,Ganymede,Male hero of Troy. Served as Zeus cup-bearer i...,1610,5262,5
5,Callisto,One of the nymphs. Daughter of king Lycaon and...,1610,4800,5
6,Amalthea,Foster-mother of Zeus,1892,200,5
7,Himalia,Nymph of the island of Rhodes. Had three sons ...,1904,170,5
8,Thebe,Name used by a few of Zeus’ daughters. It is a...,1979,90,5
9,Mimas,One of the Gigantes (giants). Son of Gaia (Earth),1789,394,6


In [89]:
sql_command = 'INSERT INTO moon(name, name_meaning, year_discovered, diameter_in_km, planet_id) VALUES'

for i in range(len(df_moon_merged)):
    moon = df_moon_merged.loc[i]
    sql_command += "('" + moon['name'] + "', '" + moon['name_meaning'] + "', " + str(moon['year_discovered']) + ', ' + str(moon['diameter_in_km']) + ', ' + str(planet['planet_id'])
    if i == len(df_moon_merged) - 1: 
        sql_command += ');'
    else:
        sql_command += '), '

print(sql_command);

INSERT INTO moon(name, name_meaning, year_discovered, diameter_in_km, planet_id) VALUES('Phobos', 'Greek god of fear and panic', 1877, 23, 17), ('Deimos', 'Greek god of dread and terror', 1877, 13, 17), ('Io', 'Ancestor to Perseus, Cadmus, Heracles, and Minos', 1610, 3630, 17), ('Europa', 'Mother of king Minos of Crete', 1610, 3138, 17), ('Ganymede', 'Male hero of Troy. Served as Zeus cup-bearer in Olympus', 1610, 5262, 17), ('Callisto', 'One of the nymphs. Daughter of king Lycaon and one of Artemis followers', 1610, 4800, 17), ('Amalthea', 'Foster-mother of Zeus', 1892, 200, 17), ('Himalia', 'Nymph of the island of Rhodes. Had three sons with Zeus: Sparteus, Kronios and Kytos', 1904, 170, 17), ('Thebe', 'Name used by a few of Zeus’ daughters. It is also the name of a city in central Greece.', 1979, 90, 17), ('Mimas', 'One of the Gigantes (giants). Son of Gaia (Earth)', 1789, 394, 17), ('Enceladus', 'One of the Gigantes (giants). Son of Gaia (Earth) and Uranus (Sky)', 1789, 502, 17), (